In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")
df.head()

sex ->(1 = erkek; 0 = kadın)

cp ->göğüs ağrısı tipi

trestbps ->dinlenme kan basıncı

chol ->mg/dl cinsinden serum kolestoral

fbs ->(açlık kan şekeri > 120 mg/dl) (1 = doğru; 0 = yanlış)

restecg ->dinlenme elektrokardiyografik sonuçları

thalack ->ulaşılan maksimum kalp atış hızı

exang ->egzersize bağlı anjina (1 = evet; 0 = hayır)

oldpeak ->dinlenmeye göre egzersizin neden olduğu ST depresyonu

slope ->irve egzersiz ST segmentinin eğimi

ca ->floroskopi ile renklendirilen büyük kapların sayısı (0-3)

thal ->3 = normal; 6 = sabit kusur; 7 = tersine çevrilebilir kusur

target -> hastalık var

In [ ]:
df.target.value_counts()

In [ ]:
sns.countplot(x="target" , data=df)
plt.show()

In [ ]:
countNoDisease = len(df[df.target == 0])
countHaveDisease = len(df[df.target ==1])
print("percentage of Patients have no disease:{} ".format((countNoDisease/len(df.target))*100) )
print("percentage of Patients have disease: {}".format((countHaveDisease/len(df.target))*100))

In [ ]:
sns.countplot(x=df.sex)
plt.show()

In [ ]:
countMale = len(df[df.sex == 1])
countFemale = len(df[df.sex == 0])
print("percentage of female Patients : {}".format(countFemale/len(df.sex)*100))
print("percentage of male Patients : {}".format(countMale/len(df.sex)*100))

In [ ]:
df.groupby("target").mean()

In [ ]:
pd.crosstab(df.age , df.target).plot(kind="bar" , figsize=(10,10))
plt.ylabel("frequency of disease" , fontsize=15)
plt.xlabel("age" , fontsize=15)
plt.title('Heart Disease Frequency for Ages',fontsize=20)
plt.show()

In [ ]:
pd.crosstab(df.sex , df.target).plot(kind="bar" , figsize=(8,8))
plt.xlabel("sex" , fontsize=15)
plt.ylabel("frequency" , fontsize=15)
plt.title("frequency of disease for sex" , fontsize=20)
plt.show()

In [ ]:
pd.crosstab(df.slope , df.target).plot(kind="bar" , figsize=(10,10))
plt.xlabel("The Slope of The Peak Exercise ST Segment" , fontsize=15)
plt.ylabel("frequency" , fontsize=15)
plt.title("Heart Disease Frequency for Slope" , fontsize=20)
plt.show()

In [ ]:
pd.crosstab(df.fbs , df.target).plot(kind="bar" , figsize=(10,10))
plt.xlabel("FBS - (Fasting Blood Sugar > 120 mg/dl) (1 = true; 0 = false)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
pd.crosstab(df.cp,df.target).plot(kind="bar",figsize=(15,6),)
plt.title('Heart Disease Frequency According To Chest Pain Type')
plt.xlabel('Chest Pain Type')
plt.ylabel('Frequency of Disease or Not')
plt.show()

Since 'cp', 'thal' and 'slope' are categorical variables we'll turn them into dummy variables.

In [ ]:
a = pd.get_dummies(df['cp'], prefix = "cp")
b = pd.get_dummies(df['thal'], prefix = "thal")
c = pd.get_dummies(df['slope'], prefix = "slope")

In [ ]:
frames = [df, a, b, c]
df = pd.concat(frames, axis = 1)
df.head()

In [ ]:
df = df.drop(columns = ['cp', 'thal', 'slope'])
df.head()

# Logistic Regression


In [ ]:
y = df.target.values
x_data = df.drop(['target'], axis = 1)

### Normalization

In [ ]:
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [ ]:
x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [ ]:
def initialize(dimension):
    
    weight = np.full((dimension,1),0.01)
    bias = 0.0
    return weight,bias

In [ ]:
def sigmoid(z):
    
    y_head = 1/(1+ np.exp(-z))
    return y_head

### Forward and Backward Propagation

In [ ]:
def forwardBackward(weight,bias,x_train,y_train):
    # Forward
    
    y_head = sigmoid(np.dot(weight.T,x_train) + bias)
    loss = -(y_train*np.log(y_head) + (1-y_train)*np.log(1-y_head))
    cost = np.sum(loss) / x_train.shape[1]
    
    # Backward
    derivative_weight = np.dot(x_train,((y_head-y_train).T))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"Derivative Weight" : derivative_weight, "Derivative Bias" : derivative_bias}
    
    return cost,gradients

In [ ]:
def update(weight,bias,x_train,y_train,learningRate,iteration) :
    costList = []
    index = []
    
    #for each iteration, update weight and bias values
    for i in range(iteration):
        cost,gradients = forwardBackward(weight,bias,x_train,y_train)
        weight = weight - learningRate * gradients["Derivative Weight"]
        bias = bias - learningRate * gradients["Derivative Bias"]
        
        costList.append(cost)
        index.append(i)

    parameters = {"weight": weight,"bias": bias}
    
    print("iteration:",iteration)
    print("cost:",cost)

    plt.plot(index,costList)
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()

    return parameters, gradients

In [ ]:
def predict(weight,bias,x_test):
    z = np.dot(weight.T,x_test) + bias
    y_head = sigmoid(z)

    y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(y_head.shape[1]):
        if y_head[0,i] <= 0.5:
            y_prediction[0,i] = 0
        else:
            y_prediction[0,i] = 1
    return y_prediction

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learningRate,iteration):
    dimension = x_train.shape[0]
    weight,bias = initialize(dimension)
    
    parameters, gradients = update(weight,bias,x_train,y_train,learningRate,iteration)

    y_prediction = predict(parameters["weight"],parameters["bias"],x_test)
    
    print("Manuel Test Accuracy: {:.2f}%".format((100 - np.mean(np.abs(y_prediction - y_test))*100)))

In [ ]:
logistic_regression(x_train,y_train,x_test,y_test,1,120)

### Logistic Regresion With Sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
accuracies = {}
lr = LogisticRegression()
lr.fit(x_train.T,y_train.T)
acc = lr.score(x_test.T,y_test.T)*100

accuracies['Logistic Regression'] = acc
print("Test Accuracy {:.2f}%".format(acc))